In [1]:
import numpy as np

In [152]:
def elimination(A, b):
    # junta a matriz A e o vetor b como uma coluna
    M = np.concatenate((A, b), axis=1)

    # para cada coluna k de 0 a n-1 (que no caso n-2), pois vamos transformar em tringular superior
    # trabalhando coluna por coluna, primeiro zerando os elementos abaixo da diagonal principal
    # na primeira coluna, depois na segunda, mas a ultima coluna não precisa ser zerada pois não
    # existem elementos abaixo da diagonal principal
    for k in range(0, M.shape[0]-1):
        # para cada linha i de k+1 a n (que no caso n-1), começa em k+1 pois zeramos somente abaixo da diagonal principal
        for i in range(k+1, M.shape[0]):
            M[i] = M[i] - M[k] * M[i, k] / M[k, k]

    # retorna a matriz A e o vetor b separadamente
    return np.split(M, [M.shape[1]-1], axis=1)

def STS(A, b):
    n = len(b)
    x = np.zeros(n)    

    # i vai variar de n-1 a 0 (por isso o n-1 no primeiro argumento, -1 porque python vai ir até 0, e o último
    # -1 diz que deve decrementar -1 de i a cada iteração), o resto será o que está a direita do x em questão,
    # por isso j vai de i+1 a n (que no caso sera n-1)
    for i in range(n-1, -1, -1):
        rest = sum([A[i, j] * x[j] for j in range(i+1, n)])
        # o .item() é para o numpy parar de reclamar
        x[i] = (b[i] - rest).item() / A[i,i]
    
    return x

def STI(A, b):
    n = len(b)
    x = np.zeros(n)

    # i vai variar de 0 a n-1, o resto será o que está a esquerda do x em questão
    # logo j vai de 0 a i-1 (é python, então tudo começa em 0 e vai até um valor antes do final)
    for i in range(n):
        rest = sum([A[i, j] * x[j] for j in range(i)])
        x[i] = (b[i] - rest).item() / A[i,i]

    return x

def LUdecomp(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            U[i, j] = A[i, j] - sum([L[i, k] * U[k, j] for k in range(i)])

        # queria poder usar i aqui, mas com j fica melhor no código, para U o i significa a linha e j a coluna,
        # para L o i significa a coluna e o j linha
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - sum([L[j, k] * U[k, i] for k in range(i)]))/U[i,i]

    L = np.identity(n) + L

    return L, U

def solveLU(A, b):
    L, U = LUdecomp(A)

    y = STI(L, b)
    x = STS(U, y)

    return x

def partialpivot(A):
    P = np.identity(A.shape[0])
    for i in range(Anew.shape[0]-1):
        biggestindex = i
        for j in range(i+1, Anew.shape[0]):
            if abs(Anew[j,0]) > abs(Anew[biggestindex,0]):
                biggestindex = j
        
        temp = np.copy(P[i,:])
        P[i,:] = P[biggestindex,:]
        P[biggestindex,:] = temp

    return P

def positivedefinite(A):
    for i in range(A.shape[0]):
        if np.linalg.det(A[:i+1,:i+1]) <= 0:
            return False

    return True

def cholesky(A):
    if not positivedefinite(A):
        return None

    G = np.zeros(A.shape)
    G[0,0] = np.sqrt(A[0,0])

    for j in range(A.shape[0]):
        if j != 0:
            G[j,j] = np.sqrt(A[j,j] - sum([G[j,k]**2 for k in range(i)]))       

        for i in range(j+1, A.shape[0]):
            if j == 0:
                G[i,j] = A[i,j] / G[0,0]
            
            else:
                G[i,j] = (A[i,j] - sum([G[i,k]*G[j,k] for k in range(j)]))/G[j,j]
        
    return G

def solvecholesky(A, b):
    G = cholesky(A)
    if G is None:
        return None

    y = STI(G, b)
    x = STS(G.T, y)

    return x

def formatacaough(A):
    for i in range(A.shape[0]):
        if len(A.shape) == 2:
            for j in range(A.shape[1]):
                print("[{0:^9}]".format("%.3g" % A[i,j]), end='')
        else:
            print("[{0:^9}]".format("%.3g" % A[i]), end='')

        print()

In [153]:
A = np.array([4., 2., -4., 2., 10., 4., -4., 4., 9.]).reshape(3,3)
print(A)
print()

G = cholesky(A)
print(G)
print(G @ G.T)

b = np.array([0., 6., 5.]).reshape(3,1)
print(solvecholesky(A, b))

[[ 4.  2. -4.]
 [ 2. 10.  4.]
 [-4.  4.  9.]]

[[ 2.  0.  0.]
 [ 1.  3.  0.]
 [-2.  2.  1.]]
[[ 4.  2. -4.]
 [ 2. 10.  4.]
 [-4.  4.  9.]]
[1. 0. 1.]
